In [107]:
# importing all necessary libraries
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point, Polygon


In [108]:
data = pd.read_csv('geonames.org_NL/NL.txt', sep = '\t', names = ['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude','longitude', 'feature class' , 'feature code', 'country code', 'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation', 'dem', 'timezone', 'modification date'])

In [109]:
data.head(3)

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
0,2743463,Den Oord,Den Oord,Oord,51.97083,5.27083,P,PPL,NL,NL,9.0,352.0,NaN,NaN,0,NaN,3,Europe/Amsterdam,2007-06-03
1,2743465,Drijberse Veld,Drijberse Veld,"Drijbersche Veld,Drijberse Veld",52.77077,6.54501,L,LCTY,NL,NaN,1.0,1731.0,NaN,NaN,0,NaN,12,Europe/Amsterdam,2017-08-16
2,2743466,Delfshavensche Schie,Delfshavensche Schie,"De Schie,Delfshavensche Schie,Delfshavense Sch...",51.90172,4.45371,H,STMC,NL,NaN,11.0,NaN,NaN,NaN,0,NaN,-3,Europe/Amsterdam,2012-01-17


In [110]:
# Select the province of Gelderland
gelderland = data[data['admin1 code'] == 3]
#
gelderland = gelderland[gelderland['feature code'] == 'PPL']

In [111]:
# Lets create a geodataframe
gdf = gpd.GeoDataFrame(
    gelderland, geometry=gpd.points_from_xy(gelderland.longitude, gelderland.latitude))

In [112]:
gdf.head(3)

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date,geometry
14,2743478,Zwolle,Zwolle,NaN,52.03167,6.65556,P,PPL,NL,NaN,3.0,1859.0,NaN,NaN,65,NaN,30,Europe/Amsterdam,2017-03-24,POINT (6.65556 52.03167)
23,2743487,Zwilbroek,Zwilbroek,NaN,52.05384,6.69250,P,PPL,NL,NaN,3.0,1859.0,NaN,NaN,0,NaN,32,Europe/Amsterdam,2011-04-19,POINT (6.69250 52.05384)
33,2743497,Zwiep,Zwiep,NaN,52.14949,6.44610,P,PPL,NL,NaN,3.0,262.0,NaN,NaN,65,NaN,15,Europe/Amsterdam,2017-03-24,POINT (6.44610 52.14949)


In [113]:
# Load a ESRI shapefile of the community of Lochem (NL)
lochemdf = gpd.read_file("lochem-shape/lochem.shp")

In [114]:
# Select only features that are within the boundaries of Lochem
# add new column to df
gdf['withinLochem'] = ""

withinLochemlist = []
for lon,lat in zip(gdf.longitude, gdf.latitude):
    pt = Point(lon, lat)
    withinLochem = pt.within(lochemdf['geometry'].values[0])
    withinLochemlist.append(withinLochem)
    
# update values in the that column, values: True/False
gdf['withinLochem'] = withinLochemlist

In [115]:
result_gdf = gdf[gdf.withinLochem==True]

In [116]:
result_gdf.head(3)

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,...,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date,geometry,withinLochem
33,2743497,Zwiep,Zwiep,NaN,52.14949,6.44610,P,PPL,NL,NaN,...,262.0,NaN,NaN,65,NaN,15,Europe/Amsterdam,2017-03-24,POINT (6.44610 52.14949),True
4707,2748381,Quatre Bras,Quatre Bras,NaN,52.18583,6.21111,P,PPL,NL,NaN,...,262.0,NaN,NaN,0,NaN,9,Europe/Amsterdam,2007-06-03,POINT (6.21111 52.18583),True
5856,2749562,Oolde,Oolde,Oolden,52.21250,6.34306,P,PPL,NL,NL,...,262.0,NaN,NaN,0,NaN,11,Europe/Amsterdam,2007-06-03,POINT (6.34306 52.21250),True


In [117]:
gjson = result_gdf.to_json()
lochemjson = lochemdf.to_json()

In [123]:
mapa = folium.Map([52.1572821,6.3015597],
                  zoom_start=11,
                  tiles='cartodbpositron')
points = folium.features.GeoJson(gjson)

# The borders of the community as well

polygon = folium.features.GeoJson(lochemjson)

mapa.add_child(polygon)

# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
for feature in points.data['features']:
        if feature['geometry']['type'] == 'Point':
            folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
                          popup = '<p>' + feature['properties']['asciiname'] +  '\nAltitude:' + str(feature['properties']['dem']) +'m</p>'
                          ).add_to(mapa)


mapa

